## Read lick events from openephys events folder

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from pathlib import Path
from datetime import datetime
import dateutil

In [ ]:
file = '/data/Clustering/SleepDeprivation/RatS/Day2NSD/lick_events/'
p = Path(file)
folders = [x for x in p.iterdir() if x.is_dir()]

for folder in folders:
    time = dateutil.parser.parse(folder.name,fuzzy=True)
    print(time)

## Create channel map for openephys in json format

In [11]:
import json
import numpy as np
import pandas as pd
import subjects

In [52]:
sess = subjects.Sd().ratNday1[0]
auxchans = sess.recinfo.auxchans
chans = np.concatenate(sess.recinfo.channelgroups)
chans = np.concatenate((chans,chans+128,auxchans+256,auxchans+256+8))
chan_map_oe = {"0":{"mapping":[int(_+1) for _ in chans],"reference":[-1]*len(chans),"enabled":['true']*len(chans)},"refs":{"channels":[]},"recording":{"channels":[]}}
with open('test_chanmap','w') as jfile:
    json.dump(chan_map_oe,jfile,indent=4)
